In [1]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F
from scipy.io import arff


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
with open('ECG5000_TRAIN.arff') as f:
  train = pd.DataFrame(arff.loadarff(f)[0])
with open('ECG5000_TEST.arff') as f:
  test = pd.DataFrame(arff.loadarff(f)[0])

In [3]:
df = train.append(test)
df = df.sample(frac=1.0)
df.shape

C:\Users\10410\AppData\Local\Temp\ipykernel_22876\2404387713.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train.append(test)


(5000, 141)

In [4]:
df[df.columns[-1]]

1001    b'1'
2086    b'1'
2153    b'1'
555     b'1'
205     b'1'
        ... 
3926    b'2'
466     b'2'
2592    b'1'
3272    b'2'
360     b'1'
Name: target, Length: 5000, dtype: object

In [5]:
CLASS_NORMAL = 1
class_names = ['Normal','R on T','PVC','SP','UB']
new_columns = list(df.columns)
new_columns[-1] = 'target'
df['target'].replace({b'1': 1, b'2': 2, b'3':3, b'4':4, b'5':5}, inplace = True)
df.columns = new_columns
df.target.value_counts()

1    2919
2    1767
4     194
3      96
5      24
Name: target, dtype: int64

In [6]:
normal_df = df[df.target == 1].drop(labels='target', axis=1)
normal_df.shape

(2919, 140)

In [7]:
anomaly_df = df[df.target != 1].drop(labels='target', axis=1)
anomaly_df.shape

(2081, 140)